In [30]:
!python --version

Python 3.9.12


In [31]:
%pip install fastf1

In [32]:
import fastf1
from fastf1 import plotting
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
from matplotlib.collections import LineCollection
from matplotlib import cm
import numpy as np
import pandas as pd

# Setup plotting
plotting.setup_mpl()

# Enable the cache
fastf1.Cache.enable_cache('cache') 

# Get rid of some pandas warnings that are not relevant for us at the moment
pd.options.mode.chained_assignment = None

In [33]:
# add monaco session, race only
monaco = fastf1.get_session(2022, 'Monaco', 'R')
monaco.load()

# load drivers performance in monaco
# only need Ferrari, Mercedes, and Redbull
mon_lec = monaco.laps.pick_driver('LEC')
mon_sai = monaco.laps.pick_driver('SAI')
mon_ham = monaco.laps.pick_driver('HAM')
mon_rus = monaco.laps.pick_driver('RUS')
mon_ver = monaco.laps.pick_driver('VER')
mon_per = monaco.laps.pick_driver('PER')

core           INFO 	Loading data for Monaco Grand Prix - Race [v2.2.9]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
d:\DH\projects\Bangkit-C22CB-Company-Based-Capstone\venv\lib\site-packages\fastf1\core.py:1310: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(new_last).reset_index(drop=True)
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['11', '55',

In [50]:
mon_lec.head()

,Time,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,...,IsPersonalBest,Compound,TyreLife,FreshTyre,LapStartTime,Team,Driver,TrackStatus,IsAccurate,LapStartDate
192,0 days 02:06:32.425000,16,NaT,1,3,0 days 02:04:10.602000,0 days 02:04:09.591000,NaT,0 days 00:01:01.525000,0 days 00:00:35.658000,...,False,WET,1.0,True,0 days 02:04:00.206000,Ferrari,LEC,2,False,2022-05-29 14:05:00.227
193,0 days 02:08:57.404000,16,0 days 00:02:24.979000,2,3,NaT,NaT,0 days 00:00:35.802000,0 days 00:01:04.157000,0 days 00:00:45.020000,...,False,WET,2.0,True,0 days 02:06:32.425000,Ferrari,LEC,4,False,2022-05-29 14:07:32.446
194,0 days 02:10:40.622000,16,0 days 00:01:43.218000,3,3,NaT,NaT,0 days 00:00:27.639000,0 days 00:00:47.939000,0 days 00:00:27.640000,...,False,WET,3.0,True,0 days 02:08:57.404000,Ferrari,LEC,1,False,2022-05-29 14:09:57.425
195,0 days 02:12:22.507000,16,0 days 00:01:41.885000,4,3,NaT,NaT,0 days 00:00:27.275000,0 days 00:00:47.325000,0 days 00:00:27.285000,...,False,WET,4.0,True,0 days 02:10:40.622000,Ferrari,LEC,1,True,2022-05-29 14:11:40.643
196,0 days 02:14:02.186000,16,0 days 00:01:39.679000,5,3,NaT,NaT,0 days 00:00:26.575000,0 days 00:00:46.700000,0 days 00:00:26.404000,...,False,WET,5.0,True,0 days 02:12:22.507000,Ferrari,LEC,1,True,2022-05-29 14:13:22.528


In [52]:
mon_lec.dtypes

Time                  timedelta64[ns]
DriverNumber                   object
LapTime               timedelta64[ns]
LapNumber                       int64
Stint                           int64
PitOutTime            timedelta64[ns]
PitInTime             timedelta64[ns]
Sector1Time           timedelta64[ns]
Sector2Time           timedelta64[ns]
Sector3Time           timedelta64[ns]
Sector1SessionTime    timedelta64[ns]
Sector2SessionTime    timedelta64[ns]
Sector3SessionTime    timedelta64[ns]
SpeedI1                       float64
SpeedI2                       float64
SpeedFL                       float64
SpeedST                       float64
IsPersonalBest                 object
Compound                       object
TyreLife                      float64
FreshTyre                      object
LapStartTime          timedelta64[ns]
Team                           object
Driver                         object
TrackStatus                    object
IsAccurate                     object
LapStartDate

In [55]:
def clean_df(df):
    '''Cleaning unneeded dataframe from the function

    Args:
        df (Dataframe): Raw dataframe from the call

    Returns:
        Dataframe: cleaned dataframe
    '''

    df = df[(df['IsAccurate'] == True)] # filter out non accurate measure
    df = df[(df['TrackStatus'] == '1')] # filter non green track
    df = df.drop([
        'Time',
        'DriverNumber',
        'Team',
        'Driver',
        'LapStartTime',
        'LapStartDate',
        'Sector1SessionTime',
        'Sector2SessionTime',
        'Sector3SessionTime'
    ], axis=1) # drop irrelevant data

    return df

cleaned_lec = clean_df(mon_lec)
cleaned_lec

,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,TrackStatus,IsAccurate
195,0 days 00:01:41.885000,4,3,NaT,NaT,0 days 00:00:27.275000,0 days 00:00:47.325000,0 days 00:00:27.285000,138.0,130.0,237.0,248.0,False,WET,4.0,True,1,True
196,0 days 00:01:39.679000,5,3,NaT,NaT,0 days 00:00:26.575000,0 days 00:00:46.700000,0 days 00:00:26.404000,147.0,135.0,239.0,251.0,False,WET,5.0,True,1,True
198,0 days 00:01:37.181000,7,3,NaT,NaT,0 days 00:00:25.558000,0 days 00:00:45.665000,0 days 00:00:25.958000,157.0,137.0,243.0,254.0,False,WET,7.0,True,1,True
199,0 days 00:01:37.426000,8,3,NaT,NaT,0 days 00:00:25.587000,0 days 00:00:45.545000,0 days 00:00:26.294000,155.0,132.0,242.0,259.0,False,WET,8.0,True,1,True
200,0 days 00:01:36.588000,9,3,NaT,NaT,0 days 00:00:25.386000,0 days 00:00:45.261000,0 days 00:00:25.941000,163.0,135.0,244.0,260.0,False,WET,9.0,True,1,True
203,0 days 00:01:35.077000,12,3,NaT,NaT,0 days 00:00:25.004000,0 days 00:00:44.191000,0 days 00:00:25.882000,170.0,133.0,244.0,264.0,False,WET,12.0,True,1,True
204,0 days 00:01:33.665000,13,3,NaT,NaT,0 days 00:00:24.878000,0 days 00:00:43.060000,0 days 00:00:25.727000,173.0,140.0,246.0,264.0,False,WET,13.0,True,1,True
205,0 days 00:01:33.314000,14,3,NaT,NaT,0 days 00:00:24.823000,0 days 00:00:42.826000,0 days 00:00:25.665000,170.0,144.0,245.0,264.0,False,WET,14.0,True,1,True
206,0 days 00:01:32.098000,15,3,NaT,NaT,0 days 00:00:24.252000,0 days 00:00:42.223000,0 days 00:00:25.623000,168.0,146.0,244.0,266.0,False,WET,15.0,True,1,True
207,0 days 00:01:32.133000,16,3,NaT,NaT,0 days 00:00:24.218000,0 days 00:00:42.725000,0 days 00:00:25.190000,172.0,150.0,245.0,264.0,False,WET,16.0,True,1,True


In [36]:
mon_ver.head()

,Time,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,...,IsPersonalBest,Compound,TyreLife,FreshTyre,LapStartTime,Team,Driver,TrackStatus,IsAccurate,LapStartDate
128,0 days 02:06:37.010000,1,NaT,1,3,0 days 02:04:19.299000,0 days 02:04:18.366000,NaT,0 days 00:01:04.033000,0 days 00:00:32.861000,...,False,WET,1.0,False,0 days 02:04:00.206000,Red Bull Racing,VER,2,False,2022-05-29 14:05:00.227
129,0 days 02:08:59.624000,1,0 days 00:02:22.614000,2,3,NaT,NaT,0 days 00:00:35.053000,0 days 00:01:05.095000,0 days 00:00:42.466000,...,False,INTERMEDIATE,NaN,True,0 days 02:06:37.010000,Red Bull Racing,VER,4,False,2022-05-29 14:07:37.031
130,0 days 02:10:45.227000,1,0 days 00:01:45.603000,3,3,NaT,NaT,0 days 00:00:28.867000,0 days 00:00:48.671000,0 days 00:00:28.065000,...,False,INTERMEDIATE,NaN,True,0 days 02:08:59.624000,Red Bull Racing,VER,1,False,2022-05-29 14:09:59.645
131,0 days 02:12:26.653000,1,0 days 00:01:41.426000,4,3,NaT,NaT,0 days 00:00:27.463000,0 days 00:00:46.871000,0 days 00:00:27.092000,...,False,INTERMEDIATE,NaN,True,0 days 02:10:45.227000,Red Bull Racing,VER,1,True,2022-05-29 14:11:45.248
132,0 days 02:14:07.571000,1,0 days 00:01:40.918000,5,3,NaT,NaT,0 days 00:00:27.345000,0 days 00:00:46.573000,0 days 00:00:27,...,False,INTERMEDIATE,NaN,True,0 days 02:12:26.653000,Red Bull Racing,VER,1,True,2022-05-29 14:13:26.674
